# Python 3 Quick Tip: The easy way to deal with file paths on Windows, Mac and Linux

One of programming’s little annoyances is that Microsoft Windows uses a backslash character between folder names while almost every other computer uses a forward slash:

```
Windows filenames:
C:\some_folder\some_file.txt

Most other operating systems:
/some_folder/some_file.txt

```

This is an accident of early 1980’s computer history. The first version of MS-DOS used the forward slash character for specifying command-line options. When Microsoft added support for folders in MS-DOS 2.0, the forward slash character was already taken so they used a backslash instead. 

Thirty-five years later, we are still stuck with this incompatibility.
If you want your Python code to work on both Windows and Mac/Linux, you’ll need to deal with these kinds of platform-specific issues. 

Luckily, Python 3 has a new module called `pathlib` that makes working with files nearly painless.

Let’s take a quick look at the different ways of handling filename paths and see how `pathlib` can make your life better!

## The Wrong Solution: Building File Paths by Hand
Let’s say you have a data folder that contains a file that you want to open in your Python program:

In [4]:
!tree data/

data/
└── MNIST
    ├── processed
    │   ├── test.pt
    │   └── training.pt
    └── raw
        ├── t10k-images-idx3-ubyte
        ├── t10k-images-idx3-ubyte.gz
        ├── t10k-labels-idx1-ubyte
        ├── t10k-labels-idx1-ubyte.gz
        ├── train-images-idx3-ubyte
        ├── train-images-idx3-ubyte.gz
        ├── train-labels-idx1-ubyte
        └── train-labels-idx1-ubyte.gz

3 directories, 10 files


In [9]:
import os
# get directory 
current_path =os.path.abspath(os.getcwd())
# get the data folder
data_folder = "/data/MNIST/"
# add the file name
file_to_open = current_path + data_folder + "readme.txt"

print(file_to_open)

# read the data
f = open(file_to_open)
print(f.read())
f.close()

/Users/caihaocui/Documents/GitHub/Data_Science_Python/01.Python/data/MNIST/readme.txt
This is MNIST data.


Notice that I’ve hardcoded the path using Unix-style forward slashes since I’m on a Mac. This will make Windows users angry.

Technically this code will still work on Windows because Python has a hack where it will recognize either kind of slash when you call open() on Windows. But even still, you shouldn’t depend on that. 

**Not all Python libraries will work if you use wrong kind of slash on the wrong operating system — especially if they interface with external programs or libraries.**

And Python’s support for mixing slash types is a Windows-only hack that doesn’t work in reverse. Using backslashes in code will totally fail on a Mac:

In [10]:
import os
# get directory 
current_path =os.path.abspath(os.getcwd())
# get the data folder
data_folder = "\\data\\MNIST\\"
# add the file name
file_to_open = current_path + data_folder + "readme.txt"

print(file_to_open)

# read the data
f = open(file_to_open)
print(f.read())
f.close()

/Users/caihaocui/Documents/GitHub/Data_Science_Python/01.Python\data\MNIST\readme.txt


FileNotFoundError: [Errno 2] No such file or directory: '/Users/caihaocui/Documents/GitHub/Data_Science_Python/01.Python\\data\\MNIST\\readme.txt'

For all these reasons and more, writing code with hardcoded path strings is the kind of thing that will make other programmers look at you with great suspicion. In general, you should try to avoid it.

## The Old Solution: Python’s os.path module

Python’s `os.path` module has lots of tools for working around these kinds of operating system-specific file system issues.

You can use `os.path.join()` to build a path string using the right kind of slash for the current operating system:


In [14]:

import os.path

current_path =os.path.abspath(os.getcwd())

data_folder = os.path.join(current_path, "data", "MNIST")

file_to_open = os.path.join(data_folder, "readme.txt")

print(file_to_open)
f = open(file_to_open)

print(f.read())

/Users/caihaocui/Documents/GitHub/Data_Science_Python/01.Python/data/MNIST/readme.txt
This is MNIST data.


This code will work perfectly on both Windows or Mac. The problem is that it’s a pain to use. Writing out `os.path.join()` and passing in each part of the path as a separate string is wordy and unintuitive.

Since most of the functions in the `os.path` module are similarly annoying to use, developers often “forget” to use them even when they know better. This leads to a lot of cross-platform bugs and angry users.

## The Better Solution: Python 3’s pathlib!

Python 3.4 introduced a new standard library for dealing with files and paths called pathlib — and it’s great!

To use it, you just pass a path or filename into a new Path() object using forward slashes and it handles the rest:

In [20]:
from pathlib import Path

current_path = Path().resolve()

data_folder = Path("./data/MNIST/")

file_to_open = current_path / data_folder / "readme.txt"

f = open(file_to_open)
print(file_to_open)
print(f.read())

f.close()

/Users/caihaocui/Documents/GitHub/Data_Science_Python/01.Python/data/MNIST/readme.txt
This is MNIST data.


Notice two things here:
- You should use forward slashes with pathlib functions. The Path() object will convert forward slashes into the correct kind of slash for the current operating system. Nice!

- If you want to add on to the path, you can use the / operator directly in your code. Say goodbye to typing out os.path.join(a, b) over and over.

And if that’s all pathlib did, it would be a nice addition to Python — but it does a lot more!

For example, we can read the contents of a text file without having to mess with opening and closing the file:

In [21]:
from pathlib import Path

data_folder = Path("./data/MNIST/")

file_to_open = data_folder / "readme.txt"

print(file_to_open.read_text())

This is MNIST data.


In fact, pathlib makes most standard file operations quick and easy:

In [23]:
from pathlib import Path

filename = Path("data/MNIST/readme.txt")

print(filename.name)
# prints "readme.txt"

print(filename.suffix)
# prints "txt"

print(filename.stem)
# prints "raw_data"

if not filename.exists():
    print("Oops, file doesn't exist!")
else:
    print("Yay, the file exists!")

readme.txt
.txt
readme
Yay, the file exists!


You can even use pathlib to explicitly convert a Unix path into a Windows-formatted path:

In [24]:
from pathlib import Path, PureWindowsPath

filename = Path("data/MNIST/readme.txt")

# Convert path to Windows format
path_on_windows = PureWindowsPath(filename)

print(path_on_windows)
# prints "source_data\text_files\raw_data.txt"

data\MNIST\readme.txt


And if you REALLY want to use backslashes in your code safely, you can declare your path as Windows-formatted and pathlib can convert it to work on the current operating system:

In [25]:
from pathlib import Path, PureWindowsPath

# I've explicitly declared my path as being in Windows format, so I can use forward slashes in it.
filename = PureWindowsPath("source_data\\text_files\\raw_data.txt")

# Convert path to the right format for the current operating system
correct_path = Path(filename)

print(correct_path)
# prints "source_data/text_files/raw_data.txt" on Mac and Linux
# prints "source_data\text_files\raw_data.txt" on Windows

source_data/text_files/raw_data.txt


If you want to get fancy, you can even use pathlib to do things like resolve relative file paths, parse network share paths and generate file:// urls. Here’s an example that will open a local file in your web browser with just two lines a code:

In [26]:
from pathlib import Path
import webbrowser

filename = Path("data/MNIST/readme.txt")

webbrowser.open(filename.absolute().as_uri())

True

This was just a tiny peak at pathlib. It’s a great replacement for lots of different file-related functionality that used to be scattered around different Python modules.

[Python3 pathlib](https://docs.python.org/3/library/pathlib.html)